In [12]:
import numpy

def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T

### Here we have ratings from eight users for six different movies: Titanic, Dirty Dancing, Die Hard, Terminator 2, Wayne's World, and Zoolander. Or in other words, two romantic films, two action films, and two comedies. Each row is a user, each column is a movie.

### The ratings are constructed so that if a user has seen both movies in one of these pairs, their ratings for the two movies are similar.

### There is no evidence in this data that anyone likes all three film genres.

In [119]:
rawdata = [
    [5,5,0,0,0,0],
    [0,0,5,5,0,0],
    [0,0,0,0,5,5],
    [0,1,5,5,5,0],
    [1,1,5,0,5,5],
    [5,5,0,5,1,1],
    [5,0,0,5,0,1],
    [5,5,5,0,1,0]
    ]
R = numpy.array(rawdata)
N = len(R)
M = len(R[0])
k = 3

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K, beta = 1.7)
prediction = numpy.dot(nP,nQ.T)
prediction

array([[ 4.23222259,  3.83253439,  3.78105921,  4.39972184,  1.39133257,
         1.23610165],
       [ 3.27463394,  2.82466635,  4.23487998,  4.26453392,  2.67219635,
         2.47359614],
       [ 1.7197877 ,  1.25182193,  4.37957831,  3.6559371 ,  4.03026854,
         3.79674908],
       [ 2.19293913,  1.75465606,  4.11031297,  3.69314779,  3.34250776,
         3.13311681],
       [ 1.43919289,  1.04406277,  3.69773792,  3.08094344,  3.4125753 ,
         3.21519477],
       [ 3.66970189,  3.3230044 ,  3.27974521,  3.8157535 ,  1.20791754,
         1.07324456],
       [ 3.6508524 ,  3.28614548,  3.44704564,  3.91714824,  1.42613146,
         1.28111094],
       [ 3.91816641,  3.52814888,  3.68647441,  4.19544321,  1.51475484,
         1.35989306]])

In [117]:
def mse(original,prediction):
    res = 0
    for i in range(original.shape[0]):
        for j in range(original.shape[1]):
            res += (original[i,j]-prediction[i,j]) ** 2
    return res**(.5)/(original.shape[0]*original.shape[1])

In [120]:
for i in numpy.linspace(1.1,2.6,11):
    nP, nQ = matrix_factorization(R, P, Q, K, beta = i)
    prediction = numpy.dot(nP,nQ.T)
    print(i, mse(R, prediction))

1.1 0.30264290347
1.25 0.2986542578
1.4 0.297212021182
1.55 0.296523649356
1.7 0.296352589084
1.85 0.296660290145
2.0 0.297428848471
2.15 0.298642327418
2.3 0.300284873271
2.45 0.302340290301
2.6 0.304791744843


### Questions

How does beta affect the results?

* try setting lambda to 0.01 (this is the default in some versions of spark)
* can you get good results? what if you increase the rank?

What happens as you increase the rank?

How sensitive are the results to the random seed?

What would happen if one movie was universally loved, or hated?

What happens if you remove some of the rating data?